<a href="https://colab.research.google.com/github/19tylermalone94/Connect_4_AI/blob/main/connect4_dataset_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

In [3]:
def get_solution(game_state: str) -> dict | None:
    # use connect4.gamesolver's server for to solve game state
    solve_url = 'https://connect4.gamesolver.org/solve'
    params = {'pos': game_state}
    session = requests.Session()
    headers = {
    'User-Agent': 'Connect 4 Machine Learning project (19tylermalone94@gmail.com) University project to train a model to learn connect 4'
    }
    response = session.get(solve_url, params=params, headers=headers)
    while response.status_code != 200:
        response = session.get(solve_url, params=params, headers=headers)
    solution = response.json()
    return solution

In [4]:
def new_board() -> np.ndarray:
    return np.zeros((6, 7), dtype=int)


def drop_piece(board: np.ndarray, col: int, piece: int) -> None:
    board[available_row(board, col)][col] = piece


def is_available(board: np.ndarray, col: int) -> bool:
    return board[0][col] == 0


def available_row(board: np.ndarray, col: int) -> int:
    for row in range(len(board) - 1, -1, -1):
        if board[row][col] == 0:
            return row
    return 0


def play(board: np.ndarray, col: int, player: int) -> bool:
    if not is_available(board, col):
        return False
    drop_piece(board, col, player)
    return True


def check_winner(board: np.ndarray) -> int | None:
    # Check horizontal locations for win
    for r in range(board.shape[0]):
        for c in range(board.shape[1] - 3):
            if board[r, c] == board[r, c+1] == board[r, c+2] == board[r, c+3] != 0:
                return board[r, c]

    # Check vertical locations for win
    for r in range(board.shape[0] - 3):
        for c in range(board.shape[1]):
            if board[r, c] == board[r+1, c] == board[r+2, c] == board[r+3, c] != 0:
                return board[r, c]

    # Check positively sloped diaganols
    for r in range(board.shape[0] - 3):
        for c in range(board.shape[1] - 3):
            if board[r, c] == board[r+1, c+1] == board[r+2, c+2] == board[r+3, c+3] != 0:
                return board[r, c]

    # Check negatively sloped diaganols
    for r in range(3, board.shape[0]):
        for c in range(board.shape[1] - 3):
            if board[r, c] == board[r-1, c+1] == board[r-2, c+2] == board[r-3, c+3] != 0:
                return board[r, c]

    return None

In [5]:
output_path = '/content/drive/MyDrive/connect-4-data/'

In [6]:
def best_move(score: list) -> str:
  max_score = min(score)
  for i in range(len(score)):
    if score[i] != 100 and score[i] > max_score:
      max_score = score[i]
  best_indexes = [i + 1 for i in range(len(score)) if score[i] == max_score]
  return str(random.choice(best_indexes))


def game_over(board: np.ndarray, seq: str) -> bool:
    return check_winner(board) is not None or len(seq) == 42


def run_perfect_games(num_games: int) -> list:
  games = []
  for _ in range(num_games):
    seq = ''
    board = new_board()
    score = get_solution(seq)['score']
    player = 1
    while True:
      if game_over(board, seq):
        break
      games.append([seq, score])
      move = best_move(score)
      seq += move
      play(board, int(move) - 1, player)
      player = 1 if player == 2 else 2
      score = get_solution(seq)['score']
      print(seq, score)
  return games

In [7]:
def random_move(seq: str) -> int:
  move = random.randint(1, 7)
  while seq.count(str(move)) >= 6:
    move = random.randint(1, 7)
  return str(move)


def run_random_games(num_games: int) -> list:
  games = []
  for _ in range(num_games):
    seq = ''
    board = new_board()
    score = get_solution(seq)['score']
    player = 1
    while True:
      if game_over(board, seq):
        break
      games.append([seq, score])
      move = random_move(seq)
      seq += move
      play(board, int(move) - 1, player)
      player = 1 if player == 2 else 2
      score = get_solution(seq)['score']
      print(seq, score)
  return games

In [8]:
def write_games(games: list, path: str) -> None:
  with open(path, 'w') as f:
    for game in games:
      f.write(game[0] + '\n')
      f.write(str(game[1]) + '\n')

perfect_games = run_perfect_games(3)
random_games = run_random_games(3)

write_games(perfect_games, output_path + 'perfect-games-data/data.csv')
write_games(random_games, output_path + 'random-games-data/data.csv')

4 [-4, -2, -2, -1, -2, -2, -4]
44 [-3, -3, -2, 1, -2, -3, -3]
444 [-4, -4, -3, -1, -3, -4, -4]
4444 [-2, -2, -2, 1, -2, -2, -2]
44444 [-1, -1, -1, -1, -1, -1, -1]
444441 [0, -3, 1, -3, 1, 0, 0]
4444413 [-16, -1, -16, -16, -1, -1, -16]
44444136 [-2, 0, 0, -3, 0, 1, 0]
444441366 [-3, -2, -2, -2, -4, -1, -3]
4444413666 [-2, -2, -2, -2, -2, 1, -3]
44444136666 [-1, -1, -1, -1, -1, -1, -3]
444441366664 [0, 1, 1, 100, 1, 1, -1]
4444413666645 [-14, -1, -14, 100, -14, -14, -14]
44444136666452 [0, 1, -2, 100, 0, -12, -12]
444441366664522 [-2, -1, -1, 100, -4, -2, -3]
4444413666645222 [-2, 1, -2, 100, -2, -2, -2]
44444136666452222 [-1, -1, -1, 100, -1, -1, -1]
444441366664522221 [1, 1, 1, 100, 1, 1, 1]
4444413666645222212 [-2, -3, -2, 100, -1, -1, -3]
44444136666452222125 [1, 1, 1, 100, 1, 1, 1]
444441366664522221256 [-2, -3, -1, 100, -4, -3, -3]
4444413666645222212563 [1, 1, 1, 100, 1, 1, 1]
44444136666452222125636 [-1, -1, -1, 100, -1, 100, -1]
444441366664522221256365 [1, 1, 1, 100, 0, 100, 1]